In [ ]:
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sns

CROP_LEFT = 94
CROP_TOP = 41
CROP_RIGHT = 13

# Обрезка изображений и аугментация данных

In [ ]:
# Указать пути до папок
input_dataset_path = "/kaggle/input/cropp-split-seismo-dataset-task-a"
cropped_dataset_path = "/kaggle/working/cropped-dataset"
augmented_dataset_path = "/kaggle/working/augmented-dataset"

# Кусок по обрезке изображений
dataset_splits = ["train", "test", "val"]
for split in dataset_splits:
    split_input_path = os.path.join(input_dataset_path, split)
    split_cropped_path = os.path.join(cropped_dataset_path, split)
    os.makedirs(split_cropped_path, exist_ok=True)
    
    for class_name in os.listdir(split_input_path):
        class_input_path = os.path.join(split_input_path, class_name)
        class_cropped_path = os.path.join(split_cropped_path, class_name)
        os.makedirs(class_cropped_path, exist_ok=True)
        
        if os.path.isdir(class_input_path):
            for image_name in os.listdir(class_input_path):
                img_path = os.path.join(class_input_path, image_name)
                img = cv2.imread(img_path)
                if img is None:
                    continue
                h, w, _ = img.shape
                # Обрезаем по вышеуказанным константам 3/4 краёв изображений
                cropped_img = img[CROP_TOP:, CROP_LEFT:w - CROP_RIGHT]
                output_path = os.path.join(class_cropped_path, image_name)
                cv2.imwrite(output_path, cropped_img)

print("Обрезка изображений завершена. Сохранено в:", cropped_dataset_path)

Аугментация (расширение) датасета

In [ ]:
for split in dataset_splits:
    split_cropped_path = os.path.join(cropped_dataset_path, split)
    split_aug_path = os.path.join(augmented_dataset_path, split)
    os.makedirs(split_aug_path, exist_ok=True)
    
    for class_name in os.listdir(split_cropped_path):
        class_cropped_path = os.path.join(split_cropped_path, class_name)
        class_aug_path = os.path.join(split_aug_path, class_name)
        os.makedirs(class_aug_path, exist_ok=True)
        
        if os.path.isdir(class_cropped_path):
            for image_name in os.listdir(class_cropped_path):
                base_name, ext = os.path.splitext(image_name)
                img_path = os.path.join(class_cropped_path, image_name)
                img = cv2.imread(img_path)
                if img is None:
                    continue
                
                # Сохраняем оригинал
                output_original = os.path.join(class_aug_path, base_name + ext)
                cv2.imwrite(output_original, img)
                
                # Аугментация 1 - горизонтальное отзеркаливание
                mirrored = cv2.flip(img, 1)
                output_mirrored = os.path.join(class_aug_path, base_name + "_mirrored" + ext)
                cv2.imwrite(output_mirrored, mirrored)
                
                # определяем разрешение изображения
                h, w, _ = img.shape
                mid = w // 2
                
                # Аугментация 2 - правая сторона отзеркаливается на левую (две правых стороны)
                right_half = img[:, mid:]
                left_mirror = cv2.flip(right_half, 1)
                double_r = np.concatenate((left_mirror, right_half), axis=1)
                output_double_r = os.path.join(class_aug_path, base_name + "_double_r" + ext)
                cv2.imwrite(output_double_r, double_r)
                
                # Аугментация 3 - левая сторона отзеркаливается на правую (две левых стороны)
                left_half = img[:, :mid]
                right_mirror = cv2.flip(left_half, 1)
                double_l = np.concatenate((left_half, right_mirror), axis=1)
                output_double_l = os.path.join(class_aug_path, base_name + "_double_l" + ext)
                cv2.imwrite(output_double_l, double_l)

print("Аугментация завершена. Сохранено в: ", augmented_dataset_path)

Показ аугментации

In [ ]:
def display_augmented_images(subfolder_root, image_base_name, ext=".png"):
    file_names = [
        image_base_name + ext,
        image_base_name + "_mirrored" + ext,
        image_base_name + "_double_r" + ext,
        image_base_name + "_double_l" + ext
    ]
    images = []
    for file in file_names:
        img_path = os.path.join(subfolder_root, file)
        img = cv2.imread(img_path)
        if img is not None:
            # Переводит BRG -> RGB для визуализации
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
        else:
            print("Image not found:", img_path)
    
    # Отображение в матплотлибе
    plt.figure(figsize=(16, 4))
    titles = ["Оригинал", "Отзеркаленное", "Две правые половины", "Две левые половины"]
    for i, image in enumerate(images):
        plt.subplot(1, 4, i + 1)
        plt.imshow(image)
        plt.title(titles[i])
        plt.axis("off")
    plt.show()
subfolder_root = "/kaggle/working/augmented-dataset/val/brak_Station"
image_base_name = "_000110"  # изображение для примеры
display_augmented_images(subfolder_root, image_base_name, ext=".png")

Удаляем плохие аугментации

In [ ]:
os.remove("/kaggle/working/augmented-dataset/test/brak_Station/_000108_double_l.png")
os.remove("/kaggle/working/augmented-dataset/test/brak_Station/_000174_double_r.png")
os.remove("/kaggle/working/augmented-dataset/train/brak_Station/_000030_double_r.png")
os.remove("/kaggle/working/augmented-dataset/train/brak_Station/_000037_double_r.png")
os.remove("/kaggle/working/augmented-dataset/train/brak_Station/_000043_mirrored.png")
os.remove("/kaggle/working/augmented-dataset/train/brak_Station/_000175_double_r.png")
os.remove("/kaggle/working/augmented-dataset/train/brak_Station/_000257_double_l.png")
# os.remove("")

Ручной перенос изображений между выборками для лучшего баланса 

In [ ]:
import shutil

source_folder = "/kaggle/working/augmented-dataset/train/brak_Station/" #откуда
destination_folder = "/kaggle/working/augmented-dataset/val/brak_Station/" #куда

# Указываем изображения для перемещения
files_to_move = ["_000257_double_l.png", "_000189.png"]

# Создаём конечную папку
os.makedirs(destination_folder, exist_ok=True)

# Перенос
for file_name in files_to_move:
    source_path = os.path.join(source_folder, file_name)
    destination_path = os.path.join(destination_folder, file_name)
    
    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
        print(f"Перемещено: {file_name}")
    else:
        print(f"Файл не найден: {file_name}")

Распределение изображений по подвыборкам

In [ ]:
import pandas as pd

base_dir = "/kaggle/working/augmented-dataset"
splits   = ["train", "val", "test"]

records = []
for split in splits:
    split_dir = os.path.join(base_dir, split)
    if not os.path.isdir(split_dir):
        print(f"Сплит не найден: {split_dir}")
        continue

    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        # Count PNG files (case‐insensitive)
        png_files = [f for f in os.listdir(class_dir)
                     if f.lower().endswith(".png")]

        records.append({
            "подвыборка":       split,
            "класс":       class_name,
            "число изображений":   len(png_files)
        })

df = pd.DataFrame(records)

for split, group in df.groupby("подвыборка"):
    print(f"== {split.upper():10s} ==")
    for _, row in group.iterrows():
        print(f"{row['класс']:<20s} : {row['число изображений']}")

display(df.sort_values(["подвыборка","класс"]).reset_index(drop=True))

# Инициализация и обучение 1й модели в каскаде

In [ ]:
import os
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns


# Гиперпараметры для модели 1 (можно менять)
num_epochs_model1 = 10 # число эпох
batch_size_model1 = 16 # размер батча 
learning_rate_model1 = 1e-3 # шаг обучения

data_dir = "/kaggle/working/augmented-dataset"
train_dir = os.path.join(data_dir, "train")
val_dir   = os.path.join(data_dir, "val")
test_dir  = os.path.join(data_dir, "test")

# Подгон формата изображений под ResNet сеть
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
}

train_dataset_orig = datasets.ImageFolder(root=train_dir, transform=data_transforms["train"])
val_dataset_orig   = datasets.ImageFolder(root=val_dir, transform=data_transforms["val"])
test_dataset_orig  = datasets.ImageFolder(root=test_dir, transform=data_transforms["test"])

print("Оригинальные классы:", train_dataset_orig.classes)


In [ ]:
class MappedDataset(torch.utils.data.Dataset):
    """
    Меняет отображения классов для выбранной модели
    """
    def __init__(self, dataset, mapping):
        self.dataset = dataset
        self.mapping = mapping
    def __getitem__(self, index):
        img, label = self.dataset[index]
        return img, self.mapping[label]
    def __len__(self):
        return len(self.dataset)

def filter_by_classes(dataset, target_class_names):
    indices = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        if dataset.classes[label] in target_class_names:
            indices.append(i)
    return Subset(dataset, indices)

In [ ]:
# Словарь для отображения оригинальных меток классов в сокращённый набор классов для модели №1
mapping1 = {}
for i, cls in enumerate(train_dataset_orig.classes):
    if cls in ["noise_DNS", "noise_Tip-off"]:
        mapping1[i] = 2  # Объединение двух шумовых классов в один — "объединённый шум"
    elif cls == "brak_Empty zone":
        mapping1[i] = 0  # Класс брака: "пустая зона"
    elif cls == "brak_Station":
        mapping1[i] = 1  # Класс брака: "станция"

# Названия классов, соответствующие новой модели №1
model1_classes = ["brak_Empty zone", "brak_Station", "noise_combined"]
print("Классы, используемые в модели №1:", model1_classes)

# Обёртка для исходных датасетов, чтобы применить отображение меток (mapping1)
train_dataset_model1 = MappedDataset(train_dataset_orig, mapping1)
val_dataset_model1   = MappedDataset(val_dataset_orig, mapping1)
test_dataset_model1  = MappedDataset(test_dataset_orig, mapping1)

# Загрузчики данных (DataLoader) для модели №1
# Здесь используется отображённый датасет, размер батча задаётся переменной batch_size_model1
train_loader_model1 = DataLoader(train_dataset_model1, batch_size=batch_size_model1, shuffle=True, num_workers=2)
val_loader_model1   = DataLoader(val_dataset_model1, batch_size=batch_size_model1, shuffle=False, num_workers=2)
test_loader_model1  = DataLoader(test_dataset_model1, batch_size=batch_size_model1, shuffle=False, num_workers=2)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = models.resnet18(pretrained=True) # инициализация resnet18 (CNN)
num_ftrs = model1.fc.in_features 
model1.fc = nn.Linear(num_ftrs, len(model1_classes)) # переопределение выходных слоёв для дообучения
model1 = model1.to(device) # перенос модели на GPU

criterion1 = nn.CrossEntropyLoss() #функция потерь
optimizer1 = optim.Adam(model1.parameters(), lr=learning_rate_model1) # оптимизатор

In [ ]:
# --- Обучение модели №1 с сохранением лучшей версии ---
best_val_acc1 = 0.0  # Лучшая достигнутая точность на валидации
train_losses_model1, val_losses_model1 = [], []  # Списки для хранения значений лосса на тренировке и валидации
train_accs_model1, val_accs_model1 = [], []      # Списки для хранения точности на тренировке и валидации

print("\nНачало обучения модели №1...")
for epoch in range(num_epochs_model1):
    # --- Фаза обучения ---
    model1.train()  # Установка модели в режим обучения
    running_loss, running_corrects = 0.0, 0  # Суммарный лосс и число правильных предсказаний

    for inputs, labels in tqdm(train_loader_model1, desc=f"Модель №1 Эпоха {epoch+1}/{num_epochs_model1} [Обучение]"):
        inputs, labels = inputs.to(device), labels.to(device)  # Перенос данных на устройство (CPU или GPU)
        optimizer1.zero_grad()  # Обнуление градиентов
        outputs = model1(inputs)  # Прямой проход модели
        loss = criterion1(outputs, labels)  # Вычисление значения функции потерь
        loss.backward()  # Обратное распространение ошибки
        optimizer1.step()  # Шаг оптимизации

        running_loss += loss.item() * inputs.size(0)  # Накопление значения функции потерь
        preds = outputs.argmax(dim=1)  # Предсказанные метки (по максимальному значению)
        running_corrects += (preds == labels).sum().item()  # Подсчёт правильных предсказаний

    epoch_loss = running_loss / len(train_dataset_model1)  # Средний лосс на эпохе
    epoch_acc  = running_corrects / len(train_dataset_model1)  # Средняя точность на эпохе
    train_losses_model1.append(epoch_loss)
    train_accs_model1.append(epoch_acc)

    # --- Фаза валидации ---
    model1.eval()  # Перевод модели в режим оценки
    val_running_loss, val_running_corrects = 0.0, 0
    with torch.no_grad():  # Отключение градиентов
        for inputs, labels in tqdm(val_loader_model1, desc=f"Модель №1 Эпоха {epoch+1}/{num_epochs_model1} [Валидация]"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model1(inputs)
            loss = criterion1(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            val_running_corrects += (preds == labels).sum().item()

    epoch_val_loss = val_running_loss / len(val_dataset_model1)  # Средний валидационный лосс
    epoch_val_acc  = val_running_corrects / len(val_dataset_model1)  # Средняя точность на валидации
    val_losses_model1.append(epoch_val_loss)
    val_accs_model1.append(epoch_val_acc)

    print(f"Эпоха {epoch+1}/{num_epochs_model1} — "
          f"Обучение: лосс = {epoch_loss:.4f}, точность = {epoch_acc:.4f} | "
          f"Валидация: лосс = {epoch_val_loss:.4f}, точность = {epoch_val_acc:.4f}")

    # --- Сохранение лучшей модели ---
    if epoch_val_acc > best_val_acc1:
        best_val_acc1 = epoch_val_acc
        torch.save(model1.state_dict(), '/kaggle/working/model1_best.pth')
        print(f"  → Новая лучшая модель! Сохранено как model1_best.pth (валидационная точность: {best_val_acc1:.4f})")

# Визуализация лоссов обучения и валидации для модели №1
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs_model1 + 1), train_losses_model1, label="Лосс на обучении")
plt.plot(range(1, num_epochs_model1 + 1), val_losses_model1, label="Лосс на валидации")
plt.xlabel("Эпоха")
plt.ylabel("Значение функции потерь")
plt.title("Модель №1: динамика функции потерь на обучении и валидации")
plt.legend()
plt.show()


In [ ]:
# --- Построение матрицы ошибок для модели №1 ---
model1.eval()  # Перевод модели в режим оценки
all_preds = []   # Предсказанные метки
all_labels = []  # Истинные метки

with torch.no_grad():  # Отключаем градиенты
    for inputs, labels in tqdm(val_loader_model1, desc="Модель №1: Матрица ошибок"):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model1(inputs)
        _, preds = torch.max(outputs, 1)  # Получаем индексы классов с максимальной вероятностью
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Вычисление матрицы ошибок
cm1 = confusion_matrix(all_labels, all_preds)

# Визуализация матрицы ошибок с подписями классов
plt.figure(figsize=(8, 6))
sns.heatmap(cm1, annot=True, fmt="d", cmap="Blues",
            xticklabels=model1_classes,
            yticklabels=model1_classes)
plt.xlabel("Предсказанный класс")
plt.ylabel("Истинный класс")
plt.title("Модель №1: Матрица ошибок на валидации")
plt.show()


# Функция отображения изображения, приведённого к исходному виду (unnormalize)
def imshow(inp):
    """Отображает изображение из тензора (обратная нормализация)."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])
    inp = (inp * std) + mean  # Обратная нормализация
    inp = np.clip(inp, 0, 1)  # Ограничение значений до диапазона [0, 1]
    plt.imshow(inp)
    plt.axis("off")


##############################
# Инференс (предсказание) для модели №1
##############################
model1.eval()  # Режим оценки модели

all_images_1 = []  # Список изображений
all_labels_1 = []  # Истинные метки
all_preds_1 = []   # Предсказанные метки

with torch.no_grad():
    for inputs, labels in val_loader_model1:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model1(inputs)
        _, preds = torch.max(outputs, 1)

        # Сохраняем изображения и метки
        for i in range(inputs.size(0)):
            all_images_1.append(inputs[i].cpu())
            all_labels_1.append(labels[i].item())
            all_preds_1.append(preds[i].item())

# Визуализация предсказаний
num_images_1 = len(all_images_1)
cols = 4  # Количество изображений в строке
rows = math.ceil(num_images_1 / cols)

plt.figure(figsize=(cols * 4, rows * 4))
for idx in range(num_images_1):
    ax = plt.subplot(rows, cols, idx + 1)
    imshow(all_images_1[idx])
    true_label = model1_classes[all_labels_1[idx]]
    pred_label = model1_classes[all_preds_1[idx]]
    ax.set_title(f"Истина: {true_label}\nПредсказание: {pred_label}", fontsize=9)
plt.tight_layout()
plt.show()

# Инициализация и обучение 2й модели в каскаде

In [ ]:
# Гиперпараметры для модели №2
num_epochs_model2 = 10  # Количество эпох обучения
batch_size_model2 = 16  # Размер батча
learning_rate_model2 = 1e-4  # Скорость обучения

# Целевые классы для модели №2 — только шумы
model2_target_names = ["noise_DNS", "noise_Tip-off"]
print("Классы модели №2:", model2_target_names)

# Фильтрация исходных датасетов: оставляем только шумовые классы
train_subset_model2 = filter_by_classes(train_dataset_orig, model2_target_names)
val_subset_model2   = filter_by_classes(val_dataset_orig, model2_target_names)
test_subset_model2  = filter_by_classes(test_dataset_orig, model2_target_names)

# Создание отображения: преобразуем оригинальные индексы классов к новым (0 и 1)
# Например: {"noise_DNS": 0, "noise_Tip-off": 1}
mapping2 = {}
for i, cls in enumerate(train_dataset_orig.classes):
    if cls == "noise_DNS":
        mapping2[i] = 0
    elif cls == "noise_Tip-off":
        mapping2[i] = 1

# Оборачивание отфильтрованных датасетов с новой схемой отображения меток
train_dataset_model2 = MappedDataset(train_subset_model2, mapping2)
val_dataset_model2   = MappedDataset(val_subset_model2, mapping2)
test_dataset_model2  = MappedDataset(test_subset_model2, mapping2)

# Загрузчики данных для модели №2
train_loader_model2 = DataLoader(train_dataset_model2, batch_size=batch_size_model2, shuffle=True, num_workers=2)
val_loader_model2   = DataLoader(val_dataset_model2, batch_size=batch_size_model2, shuffle=False, num_workers=2)
test_loader_model2  = DataLoader(test_dataset_model2, batch_size=batch_size_model2, shuffle=False, num_workers=2)

# Создание модели №2 на базе ResNet18 (предобученная версия)
model2 = models.resnet18(pretrained=True)
num_ftrs2 = model2.fc.in_features  # Размер входа в финальный полносвязный слой
model2.fc = nn.Linear(num_ftrs2, len(model2_target_names))  # Замена выходного слоя на 2-классовый
model2 = model2.to(device)  # Перенос модели на выбранное устройство

# Функция потерь и оптимизатор для модели №2
criterion2 = nn.CrossEntropyLoss()  # Кросс-энтропия для многоклассовой классификации
optimizer2 = optim.Adam(model2.parameters(), lr=learning_rate_model2)  # Адам-оптимизатор

In [ ]:
# --- Обучение модели №2 с сохранением лучшей версии ---
best_val_acc2 = 0.0  # Лучшая достигнутая точность на валидации
train_losses_model2, val_losses_model2 = [], []  # Списки лоссов на обучении и валидации
train_accs_model2, val_accs_model2 = [], []      # Списки точности на обучении и валидации

print("\nНачало обучения модели №2...")
for epoch in range(num_epochs_model2):
    # --- Фаза обучения ---
    model2.train()  # Установка режима обучения
    running_loss, running_corrects = 0.0, 0  # Суммарный лосс и количество правильных предсказаний

    for inputs, labels in tqdm(train_loader_model2, desc=f"Модель №2 Эпоха {epoch+1}/{num_epochs_model2} [Обучение]"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer2.zero_grad()  # Обнуление градиентов
        outputs = model2(inputs)  # Прямой проход
        loss = criterion2(outputs, labels)  # Вычисление функции потерь
        loss.backward()  # Обратное распространение ошибки
        optimizer2.step()  # Шаг оптимизации

        running_loss += loss.item() * inputs.size(0)  # Умножение лосса на размер батча
        preds = outputs.argmax(dim=1)  # Получение предсказаний
        running_corrects += (preds == labels).sum().item()  # Подсчёт верных предсказаний

    # Средние значения лосса и точности на обучении
    epoch_loss = running_loss / len(train_dataset_model2)
    epoch_acc  = running_corrects / len(train_dataset_model2)
    train_losses_model2.append(epoch_loss)
    train_accs_model2.append(epoch_acc)

    # --- Фаза валидации ---
    model2.eval()  # Режим оценки
    val_running_loss, val_running_corrects = 0.0, 0
    with torch.no_grad():  # Без расчёта градиентов
        for inputs, labels in tqdm(val_loader_model2, desc=f"Модель №2 Эпоха {epoch+1}/{num_epochs_model2} [Валидация]"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model2(inputs)
            loss = criterion2(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            val_running_corrects += (preds == labels).sum().item()

    # Средние значения лосса и точности на валидации
    epoch_val_loss = val_running_loss / len(val_dataset_model2)
    epoch_val_acc  = val_running_corrects / len(val_dataset_model2)
    val_losses_model2.append(epoch_val_loss)
    val_accs_model2.append(epoch_val_acc)

    # Вывод результатов эпохи
    print(f"Эпоха {epoch+1}/{num_epochs_model2} — "
          f"Обучение: лосс = {epoch_loss:.4f}, точность = {epoch_acc:.4f} | "
          f"Валидация: лосс = {epoch_val_loss:.4f}, точность = {epoch_val_acc:.4f}")

    # --- Сохранение лучшей модели ---
    if epoch_val_acc > best_val_acc2:
        best_val_acc2 = epoch_val_acc
        torch.save(model2.state_dict(), '/kaggle/working/model2_best.pth')
        print(f"  → Новая лучшая модель сохранена как model2_best.pth (валидационная точность: {best_val_acc2:.4f})")

# Визуализация лосса на обучении и валидации
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs_model2 + 1), train_losses_model2, label="Лосс на обучении")
plt.plot(range(1, num_epochs_model2 + 1), val_losses_model2, label="Лосс на валидации")
plt.xlabel("Эпоха")
plt.ylabel("Значение функции потерь")
plt.title("Модель №2: динамика функции потерь на обучении и валидации")
plt.legend()
plt.show()

In [ ]:
# --- Построение матрицы ошибок для модели №2 ---
model2.eval()  # Перевод модели в режим оценки
all_preds2 = []   # Список предсказанных меток
all_labels2 = []  # Список истинных меток

with torch.no_grad():  # Отключаем градиенты
    for inputs, labels in tqdm(val_loader_model2, desc="Модель №2: Матрица ошибок"):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model2(inputs)
        _, preds = torch.max(outputs, 1)  # Определение предсказанных классов
        all_preds2.extend(preds.cpu().numpy())
        all_labels2.extend(labels.cpu().numpy())

# Вычисление матрицы ошибок
cm2 = confusion_matrix(all_labels2, all_preds2)

# Визуализация матрицы ошибок
plt.figure(figsize=(6, 5))
sns.heatmap(cm2, annot=True, fmt="d", cmap="Blues",
            xticklabels=model2_target_names,
            yticklabels=model2_target_names)
plt.xlabel("Предсказанный класс")
plt.ylabel("Истинный класс")
plt.title("Модель №2: Матрица ошибок)

In [ ]:
inputs, labels = next(iter(val_loader_model2))
inputs = inputs.to(device)
outputs = model2(inputs)
_, preds = torch.max(outputs, 1)
plt.figure(figsize=(15, 10))
for i in range(min(inputs.size(0), 16)):
    ax = plt.subplot(4, 4, i+1)
    imshow(inputs.cpu().data[i])
    true_label = model2_target_names[labels[i]]
    pred_label = model2_target_names[preds[i]]
    ax.set_title(f"Истинный: {true_label}\nПредсказанный: {pred_label}", fontsize=9)
plt.tight_layout()
plt.show()